# MSIT Blobs

In [ ]:
import os
from surfer import Brain
%matplotlib qt4

fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'
subj_dir = os.environ["SUBJECTS_DIR"]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Surface parameters.
subject = "fscopy"
surf = "inflated"
hemi = 'lh'

## I/O parameters.
label = os.path.join(fs_dir, subject, 'label', 'april2016', 'dlpfc_5-lh.label')
overlay = os.path.join(fs_dir, subject, 'label', 'april2016', 'darpa_msit_overlay-lh.mgz')
color = '#AFFF94'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make Figure.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

brain = Brain(subject, hemi, surf)
brain.add_label(label, color=color, alpha=1, borders=3)
brain.add_overlay(overlay, min=1.301, max=5, sign='pos', name='msit')
brain.overlays['msit'].pos_bar.visible = False
brain.show_view('lateral', distance=300)

In [ ]:
brain.save_image('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/source/doris/doris_%s_%s.png' %(surf,color))

# DLPFC-5 LH Theta 

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
space = 'source'
model_name = 'revised'
label = 'dlpfc_5-lh'
analysis = 'stim'
freq = 'theta'
baseline = (-0.5, -0.1)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Load data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/%s' %space
img_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/%s' %space
info = read_csv(os.path.join(root_dir, 'afMSIT_%s_info.csv' %space))

## Load source data.
npz = np.load(os.path.join(root_dir, 'afMSIT_%s_%s_%s_%s.npz' %(space,analysis,label,freq)))
data = npz['data']
times = npz['times']

## Load cluster results.
f = os.path.join(root_dir, 'results', '%s_%s_frequency_results.csv' %(model_name, analysis))
clusters = read_csv(f)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initialize plotting
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Intialize figure.
fig = plt.figure(figsize=(15,8), dpi=90)
ax = plt.subplot2grid((1,1),(0,0))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Plot DBS Difference.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for m, color, legend in zip([0,1],['#0571b0','#ca0020'],['DBS OFF','DBS ON']):

    ix, = np.where(info.DBS==m)
    mu = data[ix].mean(axis=0)
    if analysis == 'stim': mu -= mu[(times >= baseline[0])&(times <= baseline[1])].mean()
    se = data[ix].std(axis=0) / np.sqrt(len(ix))
    ax.plot(times, mu, linewidth=7, color=color, label=legend)
    ax.fill_between(times, mu-se, mu+se, color=color, alpha=0.15)

## Plot significant clusters.
for ix in np.where((clusters.Label==label)&(clusters.Freq==freq)&
                   (clusters.Contrast=='DBS')&(clusters.FDR<0.05))[0]:
    
    y1, y2 = -1, 2.5
    tmin, tmax = clusters.loc[ix,'Tmin'], clusters.loc[ix,'Tmax']
    ax.fill_between(np.linspace(tmin,tmax,1e3), y1, y2, color='k', alpha=0.2) 
    ax.set_ylim(y1,y2)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Add Flourishes
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

ax.legend(loc=2, fontsize=30, markerscale=2, frameon=False, borderpad=0, handletextpad=0.2, handlelength=1.5)
ax.set_xlabel('Time (s)', fontsize=42)
ax.set_ylabel('Power (dB)', fontsize=40)
ax.tick_params(axis='both', which='major', labelsize=26)

y1, y2 = ax.get_ylim()
for x,s in zip([0, 0.4, 1.127],['IAPS','MSIT','Resp']): 
    ax.text(x+0.02,-0.75,s,fontsize=36)
    ax.vlines(x,y1,y2,linewidth=5, linestyle='--',alpha=0.4)
ax.set_ylim(-0.8,2.2)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.axhline(y = -0.8, linewidth=4, color='k')
ax.axvline(x = -0.5, linewidth=4, color='k')

plt.tight_layout()
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/source/doris/dlpfc_5-lh.png')
plt.close('all')
print 'Done.'

# Permutation Histograms

In [ ]:
import os
import numpy as np
import pylab as plt
import seaborn as sns
from pandas import read_csv
from scipy.ndimage import measurements
from scipy.stats import norm
sns.set(style="white")
sns.set_context(context='paper', font_scale=1.5)

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
model_name = 'revised'
analysis = 'stim'
domain = 'frequency'
freq = 'theta'

## Statistics parameters.
alpha = 0.05
ix = 0         # DBS contrast

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Load results.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source/results/revised_stim_frequency_results.csv'
results = read_csv(f)
results = results[(results.Contrast=='DBS')&(results.Freq=='theta')&(results.FDR<0.05)]
results = results[np.in1d(results.Label, ['dlpfc_4-lh','dlpfc_5-lh'])] # Limit to main VLPFC labels

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    
    
titles = ('Left DLPFC', 'Left VLPFC (Early)', 'Left VLPFC (Late)')
fig, axes = plt.subplots(1,3,figsize=(12,4),sharex=True,sharey=True)
    
for ax, label, score, title in zip(axes, results.Label, results.Score, titles):
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load files. Assemble permutations.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/source'
    results_dir = os.path.join(root_dir, model_name)
    out_dir = os.path.join(root_dir, 'results')

    ## Load null statistics.
    for n, seed in enumerate(seeds):
        npz = np.load(os.path.join(results_dir, 'afMSIT_source_%s_%s_%s_%s.npz' %(analysis, label, freq, seed)))
        if not n: permuted = npz['t_scores']
        else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)
            
    ## Prepare null statistics.
    permuted = permuted[:,ix,:]
    times = npz['times'].squeeze()

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute cluster statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Get info.
    n_shuffles, n_times  = permuted.shape

    ## Compute null cluster sums.
    null_sums = np.array([largest_cluster(permuted[m, :], threshold) for m in xrange(n_shuffles)])
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Plotting.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ax.hist(null_sums, bins=20)
    y1, y2 = ax.get_ylim()
    ax.vlines(score, y1, y2, color='k', linestyle='--', linewidth=2)
    ax.set_title(title, fontsize=24)
    if score < 100:
        ax.text(score*1.025,y2*0.925,'Cluster Score = %0.1f' %score, fontsize=14, ha='left')
    else:
        ax.text(score*0.975,y2*0.925,'Cluster Score = %0.1f' %score, fontsize=14, ha='right')
    
    
plt.tight_layout()
# plt.show()
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/doris/permutations.png')
plt.savefig('/space/sophia/2/users/EMOTE-DBS/afMSIT/plots/doris/permutations.svg')
plt.close()